### Details
#### 1. Understand the Audience and Define the Goals
Before analyzing a dataset, it's important to set a clear objective for what you aim to achieve. In this case, the focus is on understanding more about goals:
* When are goals typically scored?
* Commentators often mention "jokers," or players who are substituted late in the game. Can we observe any patterns related to this in the data?

#### 2. Quick Overview of the Data
As we examine the data, we aim to get an initial sense of its structure by considering the following:
* What columns are available, and do we understand their meaning?
* Can we determine the total row count?
* Are there missing values, and if so, where do they occur?

Here's the SQL code:

```sql
SELECT * FROM 'game_events.csv' LIMIT 100
```

| Field               | Description                                         | Type          |
|---------------------|-----------------------------------------------------|---------------|
| id                  | Refers to the unique id of the type                | integer       |
| game          | The name of the game          | string       |
| event_type          | The type of event   | string       |
| created_at          | The time when the event happened         | timestamp       |
| team_name          | The name of the team related to the event      | string       |
| player_name          | The name of the player that performed the event or to whom it happened          | string       |
| related_player_name          | The name of the player who is related to the event that `player_name` performed          | string       |
| result          | The result after the event happend    | string       |
| minute          | The game minute in which the event happened        | integer       |
| extra_minute    | The additional minutes in which the event happened (e.g. for 90+2 this would be 2)         | integer       |
| previous_player_event    | The `event_type` of the previous event that happened to the player        | string       |
| previous_player_event_at | The timestamp of the previous event that happened to the player        | timestamp       |
| seconds_after_previous_event | The seconds after the previous event happened to the player        | integer       |
| player_id          | The id of the player that performed the event or to whom it happened       | integer       |
| related_player_id          | The id of the player who is related to the event that `player_name` performed          | integer       |
| participant_id          | The id of the team related to the event          | integer       |
| fixture_id          | The id of the game the event happened in      | integer       |

#### 3. Exploratory Data Analysis
Now, we want to gain a deeper understanding of the data we're working with and identify any potential issues. A good starting point includes examining:
* The total row count
* Duplicates in the data
* Missing values
* Value distributions (e.g., spotting any unusual outliers)

#### Row counts and uniqueness

```sql
select 
	count(1) as rows,
	count(distinct id) as unique_ids
from 'game_events.csv'
```

### Summary stats
Examine the individual tables and their columns by using the `SUMMARIZE` function.

```sql
SUMMARIZE SELECT * FROM 'game_events.csv'
```

### Distributions

-- Distribution of "minute" values
```sql
select 
	minute,
	count(1) as events
from 'game_events.csv'
group by 1
```

* It appears that the majority of events occur in the second half of the game.
* There is a high number of events in the 90th minute—could these be outliers or potential data issues? 

Let’s investigate the types of events that happen during the 90th minute.

-- Events in minute 90
```sql
select 
	event_type,
	count(1) as events
from 'game_events.csv'
where minute = 90
group by 1
order by 2 desc
```

### Distribution of `extra_minute`

Create a similar plot as before, but this time focusing on the distribution of `extra_minute`.  

   ```sql
    SELECT 
        extra_minute,
        count(1) as events
    FROM 'game_events.csv'
    GROUP BY 1
   
   ```

#### 4. Data Modeling for Simplified Analysis
Structuring the data by joining tables and pre-calculating metrics for analytics can be a valuable upfront investment, saving time in the long run. Let’s create some data models that will be helpful for our analysis.

### First, let’s address a few special cases
We’ll start by going over some specific functions that we’ll be using in the process.
Now, let’s take a look at the [lpad function](https://duckdb.org/docs/sql/functions/char#lpadstring-count-character)

-- lpad function: Pads the `string` with the `character` from the left until it has `count` characters.

-- Syntax:  lpad(string, count, character)
```sql
SELECT 
	lpad(1, 1, '0'),
	lpad(1, 2, '0'),
	lpad(1, 3, '0'),
	lpad('01', 2, '0'),
	lpad(10, 2, '0'),
	lpad(100, 2, '0'), -- WARNING! THIS IS TRIMMED TO "10"
```

--> This is useful when you want to make a number string sortable

```sql
WITH data as (
	SELECT '1' as col
	UNION ALL
	SELECT '5' as col
	UNION ALL
	SELECT '10' as col
	UNION ALL
	SELECT '100' as col
)

SELECT * FROM data
ORDER BY col
```

-- Prefixing the numbers with empty strings using lpad helps fix the ordering
```sql
WITH data as (
	SELECT lpad('1', 3, ' ') as col
	UNION ALL
	SELECT lpad('5', 3, ' ') as col
	UNION ALL
	SELECT lpad('10', 3, ' ') as col
	UNION ALL
	SELECT lpad('100', 3, ' ') as col
)

SELECT * FROM data
ORDER BY col
```

Now, a useful tip for creating histograms.

-- Example: Bucket size 5 minutes
```sql
WITH buckets AS (
	SELECT DISTINCT
		minute,
		minute / 5,
		ceil(minute / 5),
		ceil(minute / 5) * 5 as ceil_bucket
	FROM 'game_events.csv'
	ORDER BY minute
)

SELECT * FROM buckets
```

### What We Aim to Improve:

* The `minute` and `extra_minute` fields alone are not very useful. We will combine them into a `game_minute` category (e.g., `90 + 5`).
* Instead of knowing the exact minute an event occurred, we need to categorize them into time ranges. Let’s group the game minutes into 5-minute intervals.

```sql
COPY (
	WITH goal_events_with_minute_string as (
		SELECT 
			*,							
			-- Use lpad for better sorting behavior (e.g. 1 -> '  1')
			lpad(minute, 3, ' ') as minute_str_sortable,
		FROM 'game_events.csv'
		WHERE event_type = 'Goal'
	)

	select 
		*,
		CASE 
			WHEN extra_minute = 0 THEN minute_str_sortable
			ELSE concat(minute_str_sortable, ' + ', extra_minute)
		END AS game_minute,
		CASE
			-- We simplify overtime and move it into a "OT" bucket
			WHEN minute > 90 THEN 'OT'
			-- For extra minutes, we use 45+, 90+, etc.
			WHEN extra_minute != 0 THEN concat(minute_str_sortable, ' +')
	 		-- Use lpad for better sorting behavior (e.g. 1 -> '  1')
			ELSE lpad(
				(ceil(minute / 5) * 5)::int,
				3,
				' '
			)
		END AS game_period_bucket
	FROM goal_events_with_minute_string
) TO 'goals.csv' (HEADER, DELIMITER ',');

SELECT * FROM 'goals.csv'
```

#### 5. In-Depth Analysis of Goals
#### When are they scored?
Count the number of goals for each `game_period_bucket` and generate a bar chart using the `goals.csv` table.

Group the data by `game_period_bucket` and tally the rows.

   ```sql
    WITH counts as (
        select 
            game_period_bucket,
            count(1) as cnt
        from 'goals.csv'
        group by 1
        order by 1
    )

    select
        *
    from counts
   
   ```



#### Fastest goal

Nedim Bajrami scored his goal within the first minute!
This particular goal was scored in just 23 seconds, making it the fastest goal ever recorded in a European Championship. 

You can see it [here](https://www.uefa.com/euro2024/news/0253-0d7dce47a88e-fed163e1fdf2-1000--who-scored-the-fastest-goals-at-uefa-european-championsh/).
      
   ```sql
    SELECT
        game,
        player_name,
        game_minute
    FROM 'goals.csv'
    ORDER BY game_minute
    LIMIT 10
   ```

### "Jokers" and their impact
Coaches often refer to certain players as "jokers." 
These are typically players they can substitute in, who are likely to score a goal. 
Let’s see how many of these goals we can actually identify.

Let's classify a goal as a "joker goal" if:

* The player's last action was a substitution.
* The goal occurred within 15 minutes (900 seconds) following the substitution.

```sql
WITH counts as (
	select 
		game_period_bucket,
		case
			-- Joker goal if player was substituted within 15 minutes prior to scoring
			when previous_player_event = 'Substitution' and seconds_after_previous_event <= 900 THEN 'joker goal'
			else 'normal goal'
		end as goal_type,
		count(1) as cnt
	from 'goals.csv'
	group by 1, 2
	order by 1
)

select
	*
from counts
```

## Utilize a Dashboard to Track EURO 2024 Results

Here’s an example dashboard created with Metabase:

[https://www.metabase.com/gallery/uefa-euro-2024-stats-dashboard](https://www.metabase.com/gallery/uefa-euro-2024-stats-dashboard?utm_source=datacamp&utm_medium=post&utm_campaign=euro-2024-dashboard)

### Which Game Had the Most Goals?

  ```sql
    SELECT
        game,
        count(1) as goals
    FROM 'goals.csv'
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10
  ```

### What is the Average Number of Goals per Game?

Try to create a bar plot that illustrates the distribution of goals per game.  
Use this template to get started:
      
  ```sql
    WITH goals_per_game as (
        select 
            fixture_id,
            count(1) as goals
        from 'goals.csv'
        group by 1
        order by 1
    )

    select
        goals,
        count(1) as cnt
    from goals_per_game
    group by 1
    order by 2
  ```